In [1]:
%matplotlib inline


# Decoding with ANOVA + SVM: meditation vs hypnosis from the "phenoscores"

Decoding on the "phenoscores" (qualitative) dataset using a feature selection, followed by an SVM.


Phenomenological ressources/datasets can be found here : 

- https://docs.google.com/spreadsheets/d/1ZNLvdWQ74B0Zv7jMfsGiZqDjLri4G0FQBkKfmwiPmVM/edit#gid=1007273088
- https://docs.google.com/spreadsheets/d/1MDJi98qrE-xq7Ral0z9ZcmcfIM-15jmu/edit#gid=2065009937
- /home/romy.beaute/projects/hypnomed/analysis/pheno_stats/AL_analysis_V02.html

## Retrieve the files of the Haxby dataset



In [2]:
from nilearn import datasets

# By default 2nd subject will be fetched
haxby_dataset = datasets.fetch_haxby()
func_img = haxby_dataset.func[0]
# print basic information on the dataset
print('Mask nifti image (3D) is located at: %s' % haxby_dataset.mask)
print('Functional nifti image (4D) is located at: %s' %func_img)

Mask nifti image (3D) is located at: /home/romy.beaute/nilearn_data/haxby2001/mask.nii.gz
Functional nifti image (4D) is located at: /home/romy.beaute/nilearn_data/haxby2001/subj2/bold.nii.gz


## Load the behavioral data



In [3]:
import pandas as pd

# Load target information as string and give a numerical identifier to each

behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
conditions = behavioral['labels']

# Restrict the analysis to faces and places
from nilearn.image import index_img
condition_mask = behavioral['labels'].isin(['face', 'house'])
conditions = conditions[condition_mask]
func_img = index_img(func_img, condition_mask)

# Confirm that we now have 2 conditions
print(conditions.unique())

# The number of the session is stored in the CSV file giving the behavioral
# data. We have to apply our session mask, to select only faces and houses.
session_label = behavioral['chunks'][condition_mask]

['face' 'house']


In [4]:
df_pheno = pd.read_csv('/home/romy.beaute/projects/hypnomed/analysis/pheno_stats/questionnaires_hypnomed.csv').dropna()
outliers = ['15','27','32','40']
sublist = df_pheno.sub_id
phenodict_control = df_pheno.set_index('sub_id').to_dict()['phenoscore_control']
phenodict_meditation = df_pheno.set_index('sub_id').to_dict()['phenoscore_meditation']
phenodict_hypnosis = df_pheno.set_index('sub_id').to_dict()['phenoscore_hypnose']
df_G1 = df_pheno[df_pheno.version=='MH']
df_G2 = df_pheno[df_pheno.version=='HM']
sublist_G1 = df_G1.sub_id
sublist_G2 = df_G2.sub_id
df_pheno 

,sub_order,sub_name,sub_id,neucose_id,version,rs_run-1,rs_run-2,rs_run-3,MH,HM,phenoscore_control,phenoscore_meditation,phenoscore_hypnose
0,70,EH070,sub-01,NEUCOSE_ELSES07842,HM,control,hypnose,meditation,0,1,-0.285714,-0.571429,1.714286
1,87,LP087,sub-02,NEUCOSE_LECPA07862,MH,control,meditation,hypnose,1,0,-0.500000,-0.500000,0.500000
2,76,HM076,sub-03,NEUCOSE_HARMA07939,MH,control,meditation,hypnose,1,0,0.250000,0.000000,2.250000
3,43,BA043,sub-04,NEUCOSE_BUEAL07925,MH,control,meditation,hypnose,1,0,0.800000,-0.600000,0.200000
4,93,BA093,sub-05,NEUCOSE_BLUAL07960,HM,control,hypnose,meditation,0,1,0.500000,0.375000,-0.125000
5,91,MP091,sub-06,NEUCOSE_MANPA07927,MH,control,meditation,hypnose,1,0,-0.600000,-0.800000,1.600000
6,67,BL067,sub-07,NEUCOSE_BETLO07962,MH,control,meditation,hypnose,1,0,31.120000,0.166667,0.666667
7,21,GM021,sub-08,NEUCOSE_MAUGI07290,HM,control,hypnose,meditation,0,1,0.750000,0.750000,0.750000
8,29,NJ029,sub-09,NEUCOSE_NICJU06244,HM,control,hypnose,meditation,0,1,0.500000,-1.000000,-0.333333
11,64,FV064,sub-12,NEUCOSE_FEYVA07961,HM,control,hypnose,meditation,0,1,0.250000,0.500000,2.000000


In [5]:
behavioral

,labels,chunks
0,rest,0
1,rest,0
2,rest,0
3,rest,0
4,rest,0
...,...,...
1447,rest,11
1448,rest,11
1449,rest,11
1450,rest,11


In [18]:
import sys
sys.path.append('/home/romy.beaute/projects/hypnomed/META/')
from scipy.io import loadmat
from helpers_gradient import *

embmat_path = '/home/romy.beaute/projects/hypnomed/data/emb_matrices'
matfile = 'control_meditation_hypnose'
# emb = np.load(embmat_path+'/group_{}_embedding.mat'.format(matfile))

b,b_emb = load_embmat(embmat_path+'/group_{}_embedding.mat'.format(matfile),show_infos=True)
b_emb.shape


outliers_indxs = [15,27,32,40]
outliers_indxs_hyp = [15,27,39]
outliers = [x-1 for x in outliers_indxs]
outliers_hyp = [x-1 for x in outliers_indxs_hyp]
# print(outliers,outliers_hyp)
# print('Subject removed (hypnose): ',outliers_hyp)

emb_con = np.delete(b_emb[:40],outliers,0)
emb_med = np.delete(b_emb[40:80],outliers,0)
emb_hyp = np.delete(b_emb[80:],outliers_hyp,0)

print(emb_con.shape,emb_med.shape,emb_hyp.shape)
embeddings = {
    'control':emb_con,
    'meditation':emb_med,
    'hypnose':emb_hyp
    }

emb_states = np.vstack([emb_con,emb_med,emb_hyp]) #embedding with all 3 states conditions
print(emb_states.shape) #control shape of the global embedding

 - shape embedding (n_subjects, n_voxels, n_dims): (119, 18715, 5)
 - n = 40 subjects
 - condition : ['control   ' 'meditation' 'hypnose   ']
 - path : /home/romy.beaute/projects/hypnomed/data/emb_matrices/group_control_meditation_hypnose_embedding.mat

(36, 18715) (36, 18715) (36, 18715)
(108, 18715)


## ANOVA pipeline with :class:`nilearn.decoding.Decoder` object

Nilearn Decoder object aims to provide smooth user experience by acting as a
pipeline of several tasks: preprocessing with NiftiMasker, reducing dimension
by selecting only relevant features with ANOVA -- a classical univariate
feature selection based on F-test, and then decoding with different types of
estimators (in this example is Support Vector Machine with a linear kernel)
on nested cross-validation.



In [ ]:
from nilearn.decoding import Decoder
# Here screening_percentile is set to 5 percent
mask_img = haxby_dataset.mask
decoder = Decoder(estimator='svc', mask=mask_img, smoothing_fwhm=4,
                  standardize=True, screening_percentile=5, scoring='accuracy')

## Fit the decoder and predict



In [ ]:
decoder.fit(func_img, conditions)
y_pred = decoder.predict(func_img)

## Obtain prediction scores via cross validation
Define the cross-validation scheme used for validation. Here we use a
LeaveOneGroupOut cross-validation on the session group which corresponds to a
leave a session out scheme, then pass the cross-validator object to the cv
parameter of decoder.leave-one-session-out For more details please take a
look at:
[Measuring prediction scores using cross-validation\](../00_tutorials/plot_decoding_tutorial.html#measuring-prediction-scores-using-cross-validation)



In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
cv = LeaveOneGroupOut()

decoder = Decoder(estimator='svc', mask=mask_img, standardize=True,
                  screening_percentile=5, scoring='accuracy', cv=cv)
# Compute the prediction accuracy for the different folds (i.e. session)
decoder.fit(func_img, conditions, groups=session_label)

# Print the CV scores
print(decoder.cv_scores_['face'])

## Visualize the results
Look at the SVC's discriminating weights using
:class:`nilearn.plotting.plot_stat_map`



In [ ]:
weight_img = decoder.coef_img_['face']
from nilearn.plotting import plot_stat_map, show
plot_stat_map(weight_img, bg_img=haxby_dataset.anat[0], title='SVM weights')

show()

Or we can plot the weights using :class:`nilearn.plotting.view_img` as a
dynamic html viewer



In [ ]:
from nilearn.plotting import view_img
view_img(weight_img, bg_img=haxby_dataset.anat[0],
         title="SVM weights", dim=-1)

Saving the results as a Nifti file may also be important



In [ ]:
weight_img.to_filename('haxby_face_vs_house.nii')